# 고급 프롬프팅을 활용한 텍스트-SQL 변환: DIN-SQL
자연어 질문을 SQL로 변환하는 고급 프롬프팅 기법 활용

---

## 권장 SageMaker 환경
Sagemaker 이미지: sagemaker-distribution-cpu

커널: Python 3

인스턴스 타입: ml.m5.large

---

## 목차

1. [의존성 설치](#step-1-install-dependencies)
1. [Athena 연결 설정](#step-2-set-up-connection-to-the-tpc-ds-data-set-in-athena)
1. [스키마 연결](#step-3-determine-schema-links)
1. [쿼리 복잡도 분류](#step-4-classify-sql-complexity)
1. [SQL 쿼리 생성](#step-5-generate-sql-query)
1. [SQL 쿼리 실행](#step-6-execute-query)
1. [결과 검증](#step-7-validate-results)
1. [자기교정](#step-8-self-correction)
1. [실험](#step-9-experiment)
1. [참고문헌](#citation)

---

## 목표
이 노트북은 자연어 질문을 해당 질문에 답하는 SQL 쿼리로 변환하는 하나의 접근 방식을 구현하는 데 도움이 되는 코드 스니펫을 제공합니다.

---

## 텍스트-SQL 문제에 대한 접근 방식
우리는 DIN-SQL 프롬프팅 전략을 구현하여 질문을 작은 부분으로 나누고, 쿼리 복잡도를 이해하며, 궁극적으로 유효한 SQL 문을 생성할 것입니다. 아래 그림에 보이는 바와 같이, 이 과정은 네 가지 주요 프롬프팅 단계로 구성됩니다:

1. 스키마 연결(Schema Linking)
2. 분류 및 분해(Classification and decomposition)
3. SQL 코드 생성(SQL code generation)
4. 자기교정(Self-correction)

이 접근 방식의 방법론과 연구 결과에 대해 더 자세히 알아보려면 전체 논문을 참고하세요: https://arxiv.org/pdf/2304.11015.pdf

![Alt text](content/din_sql_methodology.png)

### 사용 도구
SQLAlchemy, Anthropic, Amazon Bedrock SDK (Boto3), PyAthena, Jinja2

---

### 1단계: 의존성 설치

여기서 이 노트북을 실행하는 데 필요한 모든 의존성을 설치합니다. 이 모듈에서 사용하지 않을 라이브러리들의 의존성 충돌로 인해 발생할 수 있는 **다음 오류들은 무시하셔도 됩니다**:
```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.14.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.14.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.14.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.5.0 requires faiss-cpu, which is not installed.
amazon-sagemaker-jupyter-scheduler 3.0.4 requires pydantic==1.*, but you have pydantic 2.6.0 which is incompatible.
gluonts 0.13.7 requires pydantic~=1.7, but you have pydantic 2.6.0 which is incompatible.
jupyter-ai 2.5.0 requires pydantic~=1.0, but you have pydantic 2.6.0 which is incompatible.
jupyter-ai-magics 2.5.0 requires pydantic~=1.0, but you have pydantic 2.6.0 which is incompatible.
jupyter-scheduler 2.3.0 requires pydantic~=1.10, but you have pydantic 2.6.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.2 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
```

In [ ]:
!python -m ensurepip --upgrade
%pip install -qU sqlalchemy
%pip install -q "boto3~=1.34"
%pip install -qU jinja2
%pip install -qU botocore
%pip install -qU pandas
%pip install -qU PyAthena
%pip install -qU faiss-cpu

논문에서 작성된 프롬프트를 사용하는 데 도움이 되는 `din_sql` 라이브러리를 가져옵니다. 프롬프트 템플릿을 위해 Jinja를 활용했다는 점에 주목하세요.

In [ ]:
import sys

import boto3
import pandas as pd

sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl
import utilities as u

### 2단계: Athena에서 TPC-DS 데이터셋에 연결 설정

TPC-DS 데이터셋을 위한 Athena 데이터 소스 커넥터 설정과 관련된 계정 세부 정보로 다음 변수들을 초기화합니다. 이 정보들은 CloudFormation 출력에서 찾을 수 있습니다.

In [ ]:
ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME = \
    u.extract_CF_outputs("AthenaResultsS3Location", "AthenaCatalogName")
# ATHENA_RESULTS_S3_LOCATION = "<workshop bucket name>" # available in cloudformation outputs
# ATHENA_CATALOG_NAME = "<athena catalog name>" # available in cloudformation outputs
# ATHENA_RESULTS_S3_BUCKET = u.extract_s3_bucket(ATHENA_RESULTS_S3_LOCATION)
DB_NAME = "tpcds1"
ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME, DB_NAME

선택한 Bedrock 모델로 `din_sql` 클래스를 인스턴스화합니다. 이 모듈에서는 프롬프트가 Claude V2와 잘 작동하도록 특별히 맞춤화되어 있으므로, 이를 사용하겠습니다.

In [ ]:
din_sql = dsl.DIN_SQL(bedrock_model_id='anthropic.claude-v2')

위에서 입력한 정보를 사용하여 Athena에 연결을 생성합니다. 이 연결을 사용하여 생성된 SQL을 테스트할 것입니다. 또한 DIN-SQL에서 프롬프트를 보강하는 데도 사용됩니다.

In [ ]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
                       db_name=DB_NAME, 
                       s3_prefix=ATHENA_RESULTS_S3_LOCATION)

### 3단계: 스키마 연결 결정

DIN-SQL 프로세스의 첫 번째 단계는 질문에 답하기 위해 어떤 외래 키 관계가 필요한지 알아내는 것입니다. 이 작업을 위한 프롬프트가 어떻게 설계되었는지 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/schema_linking_prompt.txt.jinja

In [ ]:
return_sql = din_sql.find_fields(db_name=DB_NAME)
print(return_sql)

프롬프트 템플릿을 살펴보면, Claude와 작업할 때 결과를 개선하기 위해 몇 가지 [Anthropic 프롬프팅 모범 사례](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 사용하고 있음을 알 수 있습니다:
1. XML 태그를 사용하여 [프롬프트의 다른 부분들을 표시](https://docs.anthropic.com/claude/docs/constructing-a-prompt#mark-different-parts-of-the-prompt)합니다. 예시에서는 xml 태그와 ```sql을 사용하여 출력을 구성합니다.
2. [많은 예시를 사용](https://docs.anthropic.com/claude/docs/constructing-a-prompt#examples-optional)합니다. 이 프롬프트 기법은 Claude에게 많은 예시를 제공하는 many-shot 방법을 사용합니다.
3. [Claude에게 단계적으로 생각하도록 요청](https://docs.anthropic.com/claude/docs/ask-claude-to-think-step-by-step)합니다.
4. [역할 대화](https://docs.anthropic.com/claude/docs/roleplay-dialogue)를 사용하여 Claude가 관계형 데이터베이스 전문가 역할을 수행하도록 돕습니다.

`schema_linking_prompt_maker` 메서드에 질문과 데이터베이스 이름을 전달하여 우리의 프롬프트가 어떻게 보일지 확인해보겠습니다. 태그 사용에 주목하세요.

In [ ]:
question = "Which customer spent the most money in the web store?"

schema_links_prompt = din_sql.schema_linking_prompt_maker(question, DB_NAME)
print(schema_links_prompt)

스키마 링크에 대한 추론을 하기 전에, 어시스턴트 답변의 시작 부분을 제공하고 `llm_generation` 메서드의 `word_in_mouth` 매개변수를 활용하여 [Claude의 입에 말을 넣어주는](https://docs.anthropic.com/claude/reference/migrating-from-text-completions-to-messages#putting-words-in-claudes-mouth) 방법을 사용해보겠습니다.

In [ ]:
word_in_mouth_schema_link = f'A. Let’s think step by step. In the question "{question}", we are asked:'

이제 스키마 링크 프롬프트를 준비했으니, Claude가 어떤 결과를 낸놓는지 확인해보겠습니다.

In [ ]:
schema_links = din_sql.llm_generation(
                    schema_links_prompt,
                    stop_sequences=['</links>'],
                    word_in_mouth=word_in_mouth_schema_link
                    )
print(f"{word_in_mouth_schema_link}{schema_links}")

보시다시피, Claude는 테이블 간의 외래 키 관계를 식별하는 과정을 논리적으로 추론했습니다. 이는 Claude가 검사할 수 있도록 테이블과 그 컬럼들의 목록을 제공했기 때문입니다. `<link>` 태그를 사용하여 응답을 정리하고, DIN-SQL 방법의 다음 단계를 위해 이 목록을 저장해보겠습니다.

In [ ]:
links = u.extract_tag(schema_links+"</links>", "links")[0].strip()
links

### 4단계: SQL 복잡도 분류

프로세스의 다음 단계는 질문에 답하는 데 필요한 SQL의 복잡도를 분류하는 것입니다. 프롬프트를 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/classification_prompt.txt.jinja

여기서는 질문에 답하는 데 필요한 쿼리의 클래스를 결정하기 위한 의사결정 프레임워크를 Claude에게 제공하고 있습니다. 이는 간단한 if/then 논리를 제공함으로써 이루어집니다.

이 프롬프트가 각 클래스의 예시를 사용하여 Claude에게 의사결정 방법을 가르치는 방식을 자세히 살펴보세요. 완료되면 Claude에게 프롬프트를 보내서 이 쿼리의 복잡도를 분류해보겠습니다.

In [ ]:
word_in_mouth_classify = "A: Let’s think step by step."
classification = din_sql.llm_generation(
    prompt=din_sql.classification_prompt_maker(question, DB_NAME, links),
    word_in_mouth=word_in_mouth_classify
    )
print(f"{word_in_mouth_classify}{classification}")

Claude가 결정에 대해 생각할 여유를 준 공간을 활용하고 있는 것을 볼 수 있습니다. `<label>` 태그를 사용하여 결과를 구문 분석하고 SQL 코드 생성으로 넘어가겠습니다.

In [ ]:
predicted_class = u.extract_tag(classification, "label")[0]
predicted_class

### 5단계: SQL 쿼리 생성

질문이 준비되고, 필요한 쿼리의 복잡도가 분류되었으며, 스키마 링크가 식별되었으므로, 이제 SQL 문을 생성할 준비가 되었습니다. 그 전에 프롬프트를 살펴보겠습니다. 'NON-NESTED' 클래스는 'medium_prompt' 템플릿을 사용하므로, 이를 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/medium_prompt.txt.jinja

이러한 유형의 SQL 쿼리는 조인이 필요하므로, 이 프롬프트들은 Claude가 조인 사용법을 이해할 수 있도록 조인을 사용하는 많은 예시를 제공합니다. Claude에게 프롬프트를 보내서 어떤 결과를 생성하는지 확인해보겠습니다. 프롬프트에서 사용한 예시 종료 태그를 중지 시퀀스로 활용하여, Claude가 우리가 지시한 형식을 따를 경우 응답 생성을 중단하도록 하고 있다는 점에 주목하세요.

In [ ]:
sql_tag_start = '```sql'
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
sql_qry = din_sql.llm_generation(
                    prompt=din_sql.medium_prompt_maker(
                        test_sample_text=question, 
                        database=DB_NAME, 
                        schema_links=links,
                        sql_tag_start=sql_tag_start,
                        sql_tag_end='```'),
                    stop_sequences=['</example>'],
                    word_in_mouth=word_in_mouth_medium_prompt)
print(f"{word_in_mouth_medium_prompt}{sql_qry}")

이제 Claude가 지시사항을 따라 단계적으로 생각하고, 선택한 태그로 SQL 문을 감싸는 것을 볼 수 있습니다. 마지막 쿼리를 구문 분석해보겠습니다. 우리의 지시에 따르면 사고 연쇄 과정에서 항상 마지막이 가장 정확하기 때문입니다.

In [ ]:
SQL = sql_qry.split('```')[0]
print(f"{SQL}")

### 6단계: 쿼리 실행

쿼리를 테스트하여 결과가 우리가 예상한 것과 일치하는지, 그리고 실제로 우리 질문에 답하는지 확인해보겠습니다. SQL Alchemy 결과 집합을 반환하고 Pandas 데이터프레임을 사용하여 상호작용하는 방식으로 진행하겠습니다.

In [ ]:
result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

### 7단계: 결과 검증
웹 판매량 기준 상위 10명의 고객을 나열하는 쿼리를 제출하여 이 답변이 올바른지 확인해보겠습니다.

In [ ]:
validation_query = """
    SELECT "c"."c_customer_sk"
    , "c"."c_first_name"
    , "c"."c_last_name"
    , SUM("ws"."ws_ext_list_price") as total_sales
    FROM "customer" "c" 
    JOIN "web_sales" "ws" 
        ON "ws"."ws_bill_customer_sk" = "c"."c_customer_sk"   
    GROUP BY "c"."c_customer_sk"
    , "c"."c_first_name"
    , "c"."c_last_name"
    ORDER BY total_sales desc
    limit 10
"""
validation_set = din_sql.query(validation_query)
pd.DataFrame(validation_set)

목록 상단에 동일한 고객 SK가 보이시나요? 생성된 쿼리가 사용한 필드와 올바른 쿼리가 사용한 필드는 무엇인가요?
쿼리에서 오류가 발생했다면, LLM이 쿼리를 수정하도록 하는 자기교정 단계로 넘어가시기 바랍니다.

### 8단계: 자기교정

이는 프로세스의 마지막 단계입니다. 주어진 SQL 방언에서 코드에 잘못된 부분이 있다면 수정하도록 Claude에게 요청하여 SQL 코드를 마지막으로 한 번 더 확인합니다. 이것이 어떻게 수행되는지 지시사항을 살펴보겠습니다.

In [ ]:
!head ../libs/din_sql/prompt_templates/clean_query_prompt.txt.jinja

이제 이 템플릿을 사용하여 Athena가 기본 데이터 소스를 쿼리할 때 사용하는 "presto" 구문을 사용한 쿼리에 대한 프롬프트를 생성하겠습니다.

In [ ]:
revised_sql = din_sql.debugger_generation(
            prompt=din_sql.debugger(question, DB_NAME, SQL, sql_dialect='presto')
            ).replace("\n", " ")
print(f"{revised_sql}")

수정된 SQL이 반환되었으므로, 코드 태그를 사용하여 응답에서 구문 분석해보겠습니다.

In [ ]:
SQL = revised_sql.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

### 9단계: 실험

결과가 예상한 것과 같나요? 그렇지 않다면, 더 나은 일반화를 위해 프롬프팅을 어떻게 개선할 수 있을까요?

아래는 다른 질문에 대해 프로세스를 처음부터 끝까지 다시 실행한 것입니다.

In [ ]:
question = 'What year had the highest catalog sales?'

#get schema links
schema_links_prompt = din_sql.schema_linking_prompt_maker(question, DB_NAME)
schema_links = din_sql.llm_generation(
                    schema_links_prompt,
                    stop_sequences=['</links>'],
                    word_in_mouth=word_in_mouth_schema_link)
print(f"{word_in_mouth_schema_link}{schema_links}")
print(schema_links)
links = schema_links.split('<links>')[1].replace('\n','')

# classify and decompose
word_in_mouth_classify = "A: Let’s think step by step."
classification = din_sql.llm_generation(
    prompt=din_sql.classification_prompt_maker(question, DB_NAME, links),
    word_in_mouth=word_in_mouth_classify
    )
print(f"{word_in_mouth_classify}{classification}")
predicted_class = classification.split("<label>")[1].split("</label>")[0].strip()

# generate SQL
sql_tag_start = '```sql'
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
sql_qry = din_sql.llm_generation(
                    prompt=din_sql.medium_prompt_maker(
                        test_sample_text=question, 
                        database=DB_NAME, 
                        schema_links=links,
                        sql_tag_start=sql_tag_start,
                        sql_tag_end='```'),
                    stop_sequences=['</example>'],
                    word_in_mouth=word_in_mouth_medium_prompt
                    )
print(f"{word_in_mouth_medium_prompt}{sql_qry}")
SQL = sql_qry.split('```')[0].strip()

In [ ]:
pd.DataFrame(din_sql.query(SQL))

쿼리에서 오류가 발생했다면, 자기교정을 한 번 더 시도해보세요.

In [ ]:
# self correction
revised_sql = din_sql.debugger_generation(
            prompt=din_sql.debugger(question, DB_NAME, SQL, sql_dialect='presto')
            ).replace("\n", " ")
print(f"{revised_sql}")
SQL = revised_sql.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

# see results
result_set = pd.DataFrame(din_sql.query(SQL))
result_set

### 참고문헌
```
@article{pourreza2023din,
  title={DIN-SQL: Decomposed In-Context Learning of Text-to-SQL with Self-Correction},
  author={Pourreza, Mohammadreza and Rafiei, Davood},
  journal={arXiv preprint arXiv:2304.11015},
  year={2023}
}
논문: https://arxiv.org/abs/2304.11015
코드: https://github.com/MohammadrezaPourreza/Few-shot-NL2SQL-with-prompting
```

`hard_prompt_maker` 또는 `easy_prompt_maker`에 대한 word_in_mouth 값이 필요한 경우, 다음과 같습니다:

In [ ]:
word_in_mouth_hard_prompt = f'''A: Let's think step by step. "{question}" can be solved by knowing the answer to the following sub-question "{{sub_questions}}".
The SQL query for the sub-question "'''
word_in_mouth_easy_prompt = f"SQL: {sql_tag_start}"